<a href="https://colab.research.google.com/github/MaLuWi/guayas_store_project/blob/main/Masterschool_Week3_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
# Note: train file is very large.
# To make sure we fit RAM avaible on Google-Colab, we will load only part of the file
df_train = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/df_train2.csv') #this is the dataset after filtering out for the Guayas region and top3 familys
df_stores = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/stores.csv')
df_items = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/items.csv')
df_transactions = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/transactions.csv')
df_oil = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/oil.csv')
df_holidays_events = pd.read_csv('/content/drive/MyDrive/Time Series/datasets/holidays_events.csv')

Mounted at /content/drive


# **Checking the datasets structure**

In [ ]:
df_train.shape

In [ ]:
df_train['date'].describe()

In [ ]:
df_train.head()

In [ ]:
df_stores.head()

In [ ]:
df_holidays_events.head()

In [ ]:
df_items.head()

In [ ]:
df_oil.head()

In [ ]:
df_oil.describe()

In [ ]:
df_transactions.head()

In [ ]:
split_date = '2014-01-01'
train = df_train[df_train['date'] < split_date]
test = df_train[df_train['date'] > split_date]

In [ ]:
df_train.shape

In [ ]:
df_train.shape

In [ ]:
df_train

#Apply the Model

In [ ]:
#import the librariers
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Compute total sales per item
item_totals = df_train.groupby('item_nbr')['unit_sales'].sum().sort_values(ascending=False)
# Choose top 150 items
top_items = item_totals.head(150).index.tolist()
# Filter df_train to include only top 150 items
df_train = df_train[df_train['item_nbr'].isin(top_items)]
#I go with the most impactful 10% of items

In [ ]:
# Convert dates to datetime
df_train['date'] = pd.to_datetime(df_train['date'])

# Optimize data types in df_train
df_train = df_train.astype({
    'store_nbr': 'category',
    'item_nbr': 'category',
    'onpromotion': 'bool',
    'year': 'int16',
    'month': 'int8',
    'day': 'int8',
    'day_of_week': 'int8',
    'unit_sales_7d_avg': 'float32',
    'lag_1': 'float32',
    'lag_7': 'float32',
    'lag_14': 'float32',
    'lag_30': 'float32',
    'roll7_std': 'float32',
    'pct_chg_7d': 'float32',
    'unit_sales': 'float32'
})

# Merge datasets
df_train = df_train.merge(df_stores, on='store_nbr', how='left')
df_train = df_train.merge(df_items, on='item_nbr', how='left')

# Add is_weekend feature
df_train['is_weekend'] = df_train['day_of_week'].isin([5, 6]).astype('bool')

# Ensure categorical columns
categorical_cols = ['store_nbr', 'item_nbr', 'family', 'city', 'state', 'type_x', 'class']
for col in categorical_cols:
    if col in df_train.columns:
        df_train[col] = df_train[col].astype('category')

# Rename type_x to avoid confusion
df_train = df_train.rename(columns={'type_x': 'store_type'})

In [ ]:
split prep
split_date = '2014-01-01'
train = df_train[df_train['date'] < split_date]
test = df_train[df_train['date'] >= split_date]

In [ ]:
feature_cols = [
    'store_nbr', 'item_nbr', 'onpromotion', 'year', 'month', 'day', 'day_of_week',
    'unit_sales_7d_avg', 'lag_1', 'lag_7', 'lag_14', 'lag_30', 'roll7_std',
    'pct_chg_7d', 'is_weekend', 'city', 'state', 'store_type',
    'cluster', 'family', 'class', 'perishable'
]
# Ensure only existing columns are included
feature_cols = [col for col in feature_cols if col in df_train.columns]
X_train = train[feature_cols]
y_train = train['unit_sales']
X_test = test[feature_cols]
y_test = test['unit_sales']

In [ ]:
models = {
    'XGBoost': {
        'model': xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True),
        'param_grid': {
            'eta': [0.01, 0.1, 0.3],
            'max_depth': [3, 5, 7, 10],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'n_estimators': [100, 200, 500]
        }
    }
}

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
best_models = {}
best_scores = {}
for model_name, model_info in models.items():
    print(f"Training {model_name}...")
    random_search = RandomizedSearchCV(
        estimator=model_info['model'],
        param_distributions=model_info['param_grid'],
        cv=tscv,
        n_iter=20,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    random_search.fit(X_train, y_train)
    best_models[model_name] = random_search.best_estimator_
    best_scores[model_name] = random_search.best_score_
    print(f"Best parameters for {model_name}: {random_search.best_params_}")
    print(f"Best CV score (MSE) for {model_name}: {-random_search.best_score_}")

best_model_name = max(best_scores, key=best_scores.get)
best_model = best_models[best_model_name]
print(f"Best model: {best_model_name} with CV score (MSE): {-best_scores[best_model_name]}")

In [ ]:
y_pred = best_model.predict(X_test)
print(f"Evaluation metrics for {best_model_name}:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
i = 30
plt.figure(figsize=(14, 7))
plt.plot(test['date'][:i], y_test.values[:i], label='Actual Sales', marker='o')
plt.plot(test['date'][:i], y_pred[:i], label='Predicted Sales', marker='x', color='red')
plt.title(f'Actual vs Predicted Sales using {best_model_name}')
plt.xlabel('Date')
plt.ylabel('Unit Sales')
plt.legend()
plt.grid(True)
plt.savefig('/content/drive/MyDrive/demand_forecast_plot.png')

In [ ]:
def generate_future_data(store_nbr, item_nbr, start_date, days_ahead, df_train, df_stores, df_items):
    future_dates = pd.date_range(start=start_date, periods=days_ahead, freq='D')
    future_df = pd.DataFrame({
        'date': future_dates,
        'store_nbr': store_nbr,
        'item_nbr': item_nbr,
        'onpromotion': False,
        'year': future_dates.year,
        'month': future_dates.month,
        'day': future_dates.day,
        'day_of_week': future_dates.dayofweek
    })

    # Merge store and item info
    future_df = future_df.merge(df_stores, on='store_nbr', how='left')
    future_df = future_df.merge(df_items, on='item_nbr', how='left')

    # Add lag and rolling features
    last_data = df_train[(df_train['store_nbr'] == store_nbr) & (df_train['item_nbr'] == item_nbr)].tail(30)
    future_df['lag_1'] = np.pad(last_data['unit_sales'].values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['lag_7'] = np.pad(last_data['unit_sales'].shift(6).fillna(method='bfill').values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['lag_14'] = np.pad(last_data['unit_sales'].shift(13).fillna(method='bfill').values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['lag_30'] = np.pad(last_data['unit_sales'].shift(29).fillna(method='bfill').values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['unit_sales_7d_avg'] = np.pad(last_data['unit_sales'].rolling(window=7).mean().fillna(method='bfill').values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['roll7_std'] = np.pad(last_data['unit_sales'].rolling(window=7).std().fillna(method='bfill').values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]
    future_df['pct_chg_7d'] = np.pad(last_data['unit_sales'].pct_change(periods=7).fillna(0).values, (0, days_ahead - len(last_data)), mode='edge')[:days_ahead]

    # Add is_weekend
    future_df['is_weekend'] = future_df['day_of_week'].isin([5, 6]).astype('bool')

    # Ensure categorical columns
    categorical_cols = ['store_nbr', 'item_nbr', 'family', 'city', 'state', 'store_type', 'class']
    for col in categorical_cols:
        if col in future_df.columns:
            future_df[col] = future_df[col].astype('category')

    # Ensure feature order matches training
    return future_df[feature_cols]

In [ ]:
pickle_path = '/content/drive/MyDrive/models/best_demand_forecast_model.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(best_model, f)
print(f"Best model saved as pickle file at: {pickle_path}")

In [ ]:
import os

model_dir = '/content/drive/MyDrive/models'
os.makedirs(model_dir, exist_ok=True)


In [ ]:
import pickle

model_path = os.path.join(model_dir, f'{best_model_name}_forecast_model.pkl')
with open(model_path, 'wb') as f:
    pickle.dump(best_model, f)

print(f"Model saved at: {model_path}")
